In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [49]:
data = pd.read_csv('movie_bd_v5.csv')
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


In [50]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [51]:
answers = {}  # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
# ...

# получаем новый стобец "прибыль"
data['profit'] = data['revenue'] - data['budget']
data['release_month'] = pd.to_datetime(
    data['release_date']).dt.month  # добавим столбец с месяцем выпуска


def counting_words(s):
    ''' Функция получает список слов из строковой переменной 

    и увеличивает их счетчик.  

    '''

    words = s.split('|')
    for word in words:
        # count_words - глобальная переменная collections.Counter()
        count_words[word] += 1


def counting_months(s):
    # получаем список слов из строковой переменной и увеличиваем их счетчик
    words = s.split('/')
    # берем первый элемент списка, что в данном формате соответствует месяцу
    word = words[0]
    # count_words - глобальная переменная collections.Counter()
    count_months[word] += 1


def unfolding_data_old(dataframe, column_names):
    ''' Функция получает на вход датафрейм и список названий столбцов в которых могут

    быть строковые данные с разделителем '|', на выходе получаем датафрейм, в котором

    первоначальные n слов (разделенных '|') в столбцах column_names  разворачиваются 

    по этим столбцам в n строк, при этом данные по этим строкам в остальных столбцах 

    дублируются. Таким образом, данные подготавливаются к дальнейшим группировкам.

    Это первая версия выполняется долго и приведена как первоначальный вариант 

    для unfolding_data(dataframe, column_names ).  

    '''

    df = pd.DataFrame(columns=dataframe.columns)
    rows_num = dataframe.shape[0]

    for column_name in column_names:
        for i in range(rows_num):
            words = dataframe.loc[i, column_name].split('|')
            for word in words:
                s = dataframe.loc[i, :].copy()
                s[column_name] = word
                # data_new.append(s,ignore_index=True) - функция append оказалась слишком долгой
                df.loc[len(df), :] = s
        rows_num = df.shape[0]

    return df


def unfolding_data(dataframe, column_names):
    ''' Функция получает на вход датафрейм и список названий столбцов в которых могут

    быть строковые данные с разделителем '|', на выходе получаем датафрейм, в котором

    первоначальные n слов (разделенных '|') в столбцах column_names  разворачиваются 

    по этим столбцам в n строк, при этом данные по этим строкам в остальных столбцах 

    дублируются. Таким образом, данные подготавливаются к дальнейшим группировкам.

    '''

    for column_name in column_names:
        # создадим столбец с развернутыми данными
        s = dataframe[column_name].str.split('|',  expand=True).stack()
        # уберем мультииндекс и присвоим имя для использования join()
        s.index = s.index.droplevel(-1)
        s.name = column_name
        dataframe = dataframe.drop([column_name], axis=1)
        dataframe = dataframe.join(s)
    return dataframe


def combinations_to_series(s):
    ''' Функция добавляет строку c строковым объединением имен актеров 

    в глобальной переменной casts являющейся pd.Series

    '''
    # получим список комбинаций пар актеров для каждого фильма
    c = list(combinations(sorted(s.split('|')), 2))
    for i in range(len(c)):
        # для каждой пары получим строковое объединение имен актеров
        c[i] = c[i][0] + '&' + c[i][1]
        casts[len(casts)] = c[i]

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [52]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример:
#answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
# +
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [53]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [54]:
# можно добавлять разные варианты решения
data.sort_values('budget', ascending=False)[
    ['original_title', 'imdb_id']].head(1)

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [55]:
# думаю логику работы с этим словарем вы уже поняли,
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'  # +

In [56]:
data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [57]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'  # +

In [58]:
data[data.runtime == data.runtime.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [59]:
answers['4'] = '2. 110'  # +
round(data.runtime.mean(), 0)

110.0

# 5. Каково медианное значение длительности фильмов? 

In [60]:
answers['5'] = '1. 107'  # +
round(data.runtime.median(), 0)

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [61]:
answers['6'] = '5. Avatar (tt0499549)'  # +
# лучше код получения столбца profit вынести в Предобработку что в начале

data[data.profit == data.profit.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [62]:
answers['7'] = '5. The Lone Ranger (tt1210819)'  # +

data[data.profit == data.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [63]:
answers['8'] = '1. 1478'  # +

data[data.profit > 0].original_title.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [64]:
answers['9'] = '4. The Dark Knight (tt0468569)'  # +

data[(data.release_year == 2008)].sort_values(
    'revenue', ascending=False)[['original_title', 'imdb_id']].head(1)

,original_title,imdb_id
599,The Dark Knight,tt0468569


ВАРИАНТ 2

In [65]:
df = data[(data.release_year == 2008)]
df[df.revenue == df.revenue.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [66]:
answers['10'] = '5. The Lone Ranger (tt1210819)'  # +

data[(2012 <= data.release_year) & (data.release_year <= 2014)].sort_values(
    'profit')[['original_title', 'imdb_id', 'profit']].head(1)

,original_title,imdb_id,profit
1245,The Lone Ranger,tt1210819,-165710090


# 11. Какого жанра фильмов больше всего?

In [67]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
answers['11'] = '3. Drama'  # +

data["genres"].str.split('|',  expand=True).stack().value_counts().keys()[0]

'Drama'

ВАРИАНТ 2

In [68]:
pd.Series(data['genres'].str.cat(sep='|').split('|')).value_counts().keys()[0]

'Drama'

#### ВАРИАНТ 3

In [69]:
count_words = Counter()
data['genres'].apply(counting_words)
count_words.most_common()[0][0]

'Drama'

#### ВАРИАНТ 4

In [70]:
# Внимание код выполняется долго - был самый первый вариант
df = unfolding_data_old(data, ['genres'])
df['genres'].value_counts().keys()[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [71]:
answers['12'] = '1. Drama'  # +

data[data.profit > 0]["genres"].str.split(
    '|',  expand=True).stack().value_counts().keys()[0]

'Drama'

ВАРИАНТ 2

In [72]:
count_words = Counter()
data[data.profit > 0]['genres'].apply(counting_words)
count_words.most_common()[0][0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [73]:
answers['13'] = '5. Peter Jackson'  # +

# получим датафрейм с развернутыми данными
# по каждому режиссеру в отдельности
data_director = unfolding_data(data, ['director'])

grouped_by_director = data_director.groupby(['director']).revenue.sum()
grouped_by_director[grouped_by_director == grouped_by_director.max()].keys()[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [74]:
answers['14'] = '3. Robert Rodriguez'  # +

grouped_by_director = data_director[data_director['genres'].str.contains(
    'Action', regex=False)].groupby(['director']).genres.count()
grouped_by_director[grouped_by_director == grouped_by_director.max()].keys()[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [75]:
answers['15'] = '3. Chris Hemsworth'  # +

# получим датафрейм с развернутыми данными
# по каждому актеру в отдельности
data_cast = unfolding_data(data, ['cast'])
grouped_by_Actor = data_cast[data_cast.release_year == 2012].groupby([
                                                                     'cast']).revenue.sum()
print(*grouped_by_Actor[grouped_by_Actor == grouped_by_Actor.max()].keys())

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [76]:
answers['16'] = '3. Matt Damon'  # +

mean_budget = data['budget'].mean()
data_cast[data_cast.budget > mean_budget].cast.value_counts().keys()[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [77]:
answers['17'] = '2. Action'  # +

count_words = Counter()
data[data['cast'].str.contains('Nicolas Cage')]['genres'].apply(counting_words)
count_words.most_common()[0][0]

'Action'

ВАРИАНТ 2

In [78]:
count_words = Counter()
data_cast[data_cast['cast'] == 'Nicolas Cage']['genres'].apply(counting_words)
count_words.most_common()[0][0]

'Action'

ВАРИАНТ 3

In [79]:
# получим датафрейм с развернутыми данными
# по каждому актеру и по каждому жанру в отдельности
data_cast_genres = unfolding_data(data, ['cast', 'genres'])
data_cast_genres[data_cast_genres['cast'] ==
                 'Nicolas Cage']['genres'].value_counts().keys()[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [80]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'  # +

data[data['production_companies'].str.contains('Paramount Pictures')].sort_values([
    'profit'])['original_title'].values[0]

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [81]:
answers['19'] = '5. 2015'  # +

data.groupby(['release_year']).revenue.sum(
).sort_values(ascending=False).keys()[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [82]:
answers['20'] = '1. 2014'  # +

data[data['production_companies'].str.contains('Warner Bros')].groupby(['release_year']).profit.sum(
).sort_values(ascending=False).keys()[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [83]:
answers['21'] = '4. Сентябрь'  # +

data.loc[:, 'release_month'].value_counts().keys()[0]

9

ВАРИАНТ 2

In [84]:
# без введения столбца release_month
count_months = Counter()
data['release_date'].apply(counting_months)
count_months.most_common()[0][0]

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [85]:
answers['22'] = '2. 450'  # +

s = data.loc[:, 'release_month'].value_counts()
s[s.index.isin([6, 7, 8])].sum()

450

ВАРИАНТ 2

In [86]:
# без введения столбца release_month
count_months = Counter()
data['release_date'].apply(counting_months)
sum = 0
for m in ['6', '7', '8']:
    sum += count_months[m]
print(sum)

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [87]:
answers['23'] = '5. Peter Jackson'  # +

count_words = Counter()
data[data.release_month.isin([12, 1, 2])]['director'].apply(counting_words)
count_words.most_common()[0][0]

'Peter Jackson'

ВАРИАНТ 2

In [88]:
# развернем данные по каждому режисеру в отдельности
data_director = unfolding_data(data, ['director'])
data_director[data_director.release_month.isin(
    [12, 1, 2])]['director'].value_counts().keys()[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [89]:
answers['24'] = '5. Four By Two Productions'  # +

df = data.copy()  # чтобы не изменять исходный датафрейм
# получим столбец с количеством слов в описаний фильмов
df['title_len'] = df['original_title'].apply(lambda s: len(s))
# развернем данные по каждой студии в отдельности
data_companies = unfolding_data(df, ['production_companies'])
data_companies.groupby(['production_companies']).title_len.mean(
).sort_values(ascending=False).keys()[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [90]:
answers['25'] = '3. Midnight Picture Show'  # +

df = data.copy()  # чтобы не изменять исходный датафрейм
# получим столбец с количеством слов в описаний фильмов
df['overview_len'] = df['overview'].apply(lambda s: len(s.split()))
# развернем данные по каждой студии в отдельности
data_companies = unfolding_data(df, ['production_companies'])
data_companies.groupby(['production_companies']).overview_len.mean(
).sort_values(ascending=False).keys()[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [91]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'  # +

# получим список фильмов входящих в один процент по рейтингу
print(data[data.vote_average >
           data.vote_average.quantile(0.99)]['original_title'].values)

['Inside Out' 'Room' 'Interstellar' 'Guardians of the Galaxy'
 'The Imitation Game' 'Gone Girl' 'The Grand Budapest Hotel' 'Inception'
 'The Dark Knight' 'The Pianist'
 'The Lord of the Rings: The Return of the King' 'The Wolf of Wall Street'
 '12 Years a Slave' 'Memento']


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [92]:
answers['27'] = '5. Daniel Radcliffe&Rupert Grint'  # +

# Внимание! код выполняется долго: apply - не самый быстрый метод:((

# получим объект Series из строковых объединений пар имен актеров
casts = pd.Series([], dtype=str)
data['cast'].apply(combinations_to_series)

s = casts.value_counts()
print(*s[s.values == s.values.max()].keys(), sep=', ')

Daniel Radcliffe&Emma Watson, Daniel Radcliffe&Rupert Grint, Emma Watson&Rupert Grint


# Submission

In [93]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe&Rupert Grint'}

In [94]:
# и убедиться что ни чего не пропустил)
len(answers)

27